<a href="https://colab.research.google.com/github/AliyaK05/mycaptain-programs/blob/main/Text%20Generation%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data and opening data in form of text file - taken from project gutenberg site
import requests
result=requests.get('https://www.gutenberg.org/cache/epub/84/pg84.txt')
file = result.text

In [4]:
# import dependencies
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [5]:
# tokenization-breaking up a stream of text into words,phrases, symbols or meaninful elements
# standardization
def tokenize_words(input):
  input = input.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return "".join(filtered)

p_inputs = tokenize_words(file)


In [6]:
# chars to numbers
chars = sorted(list(set(p_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [7]:
# check if words to chars or chars to num(?!) has worked?
input_len = len(p_inputs)
vocab_len = len(chars)
print("Total number of characters: ", input_len)
print("Total vocab: ", vocab_len)

Total number of characters:  233002
Total vocab:  42


In [8]:
# seq length
seq_length = 100
x_data = []
y_data = []

In [9]:
# loop through the sequence
for i in range(0, input_len - seq_length, 1):
  in_seq = p_inputs[i:i + seq_length]
  out_seq = p_inputs[i + seq_length]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 232902


In [10]:
# convert input sequence to np array and so on
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [11]:
# one-hot encoding
y = np_utils.to_categorical(y_data)

In [12]:
# creating the model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [13]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# saving weights
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [15]:
# fit model and let it train
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
910/910 [==============================] - ETA: 0s - loss: 2.9363
Epoch 1: loss improved from inf to 2.93635, saving model to model_weights_saved.hdf5
910/910 [==============================] - 3671s 4s/step - loss: 2.9363
Epoch 2/4
910/910 [==============================] - ETA: 0s - loss: 2.9153
Epoch 2: loss improved from 2.93635 to 2.91525, saving model to model_weights_saved.hdf5
910/910 [==============================] - 3634s 4s/step - loss: 2.9153
Epoch 3/4
910/910 [==============================] - ETA: 0s - loss: 2.9120
Epoch 3: loss improved from 2.91525 to 2.91198, saving model to model_weights_saved.hdf5
910/910 [==============================] - 3632s 4s/step - loss: 2.9120
Epoch 4/4
910/910 [==============================] - ETA: 0s - loss: 2.8886
Epoch 4: loss improved from 2.91198 to 2.88858, saving model to model_weights_saved.hdf5
910/910 [==============================] - 3670s 4s/step - loss: 2.8886


In [16]:
# recompile model with the saved weights
filename = 'model_weights_saved.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
# output of the model back into characters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [18]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\".".join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
p".p".e".a".c".e".t".h".i".n".k".s".s".u".r".e".l".y".t".h".i".n".k".t".h".u".s".f".a".r".e".w".e".l".l".s".p".r".a".n".g".c".a".b".i".n".w".i".n".d".o".w".s".a".i".d".u".p".o".n".i".c".e".r".a".f".t".l".a".y".c".l".o".s".e".v".e".s".s".e".l".s".o".o".n".b".o".r".n".e".a".w".a".y".w".a".v".e".s".l "


In [19]:
# generate the text
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seg_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]


eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee